<a href="https://colab.research.google.com/github/Tixonmavrin/Zindi-Zimnat-Insurance-Recommendation-Challenge/blob/master/Baseline1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Zimnat Insurance Recommendation Challenge
Can you predict which insurance products existing clients will want next?
###Baseline with catboost

### Get data

In [21]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import sklearn

import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn import preprocessing
from sklearn import metrics
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
from xgboost.sklearn import XGBClassifier 
from mlxtend.feature_selection import SequentialFeatureSelector
import warnings
warnings.filterwarnings("ignore")

# Classifiers
from catboost import CatBoostClassifier

# Model selection
from sklearn.model_selection import StratifiedKFold

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer
from time import time

In [3]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
submission = pd.read_csv('SampleSubmission.csv')

### New format

In [4]:
X_train = []
X_train_columns = train.columns
c = 0
for v in train.values:
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for i in index:
        c+=1
        for k in range(len(binary)):
            if k == i:
                binary_transformed = list(copy.copy(binary))
                binary_transformed[i] = 0
                X_train.append(list(info) + binary_transformed + [X_train_columns[8+k]] + [c])

X_train = pd.DataFrame(X_train)
X_train.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'ID2']

In [5]:
X_test = []
true_values = []
c = 0
for v in test.values:
    c += 1
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    X_test.append(list(info) + list(binary) + [c])
    for k in test.columns[8:][index]:
        true_values.append(v[0] + ' X ' + k)

X_test = pd.DataFrame(X_test)
X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'ID2']

### Transform data

In [6]:
features_train = []
features_test = []
columns = []

append_features = ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 
'N2MW', 'AHXO','BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 
'ECY3', 'ID', 'ID2', 'join_date', 'sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',
'birth_year']
for v in append_features:
    features_train.append(X_train[v].values.reshape(-1, 1))
    features_test.append(X_test[v].values.reshape(-1, 1))
    columns.append(np.array([v]))

y_train = X_train[['product_pred']]

In [7]:
features_train = np.concatenate(features_train, axis=1)
features_test = np.concatenate(features_test, axis=1)
columns = np.concatenate(np.array(columns))

X_train = pd.DataFrame(features_train)
X_train.columns = columns
X_test = pd.DataFrame(features_test)
X_test.columns = columns

### New features

In [8]:
X_train['date1'] = X_train['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
X_train['date2'] = X_train['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
X_train['date3'] = X_train['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
X_train.drop('join_date', axis=1, inplace=True)

X_test['date1'] = X_test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
X_test['date2'] = X_test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
X_test['date3'] = X_test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
X_test.drop('join_date', axis=1, inplace=True)

X_train['date_diff'] = X_train['date3'] - X_train['birth_year']
X_test['date_diff'] = X_test['date3'] - X_test['birth_year']

### Change types

In [9]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data = X_train.append(X_test)
for v in ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',]:
    data.loc[:,v] = le.fit_transform(data.loc[:,v])
X_train = data[:X_train.shape[0]]
X_test = data[-X_test.shape[0]:]

In [11]:
le.fit(y_train.iloc[:,0])
y_train = pd.DataFrame(le.transform(y_train.iloc[:,0]))
y_train.columns = ['target']

In [12]:
pd.set_option('display.max_columns', None)
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,1/8/2019,M,M,1990,1X1H,BP09,56SI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,1/8/2019,M,M,1990,748L,NO3L,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [13]:
pd.set_option('display.max_columns', None)
X_train.head()

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,date1,date2,date3,date_diff
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4WKQSBB,1,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4WKQSBB,2,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,CP5S02H,3,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,CP5S02H,4,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,2YKDILJ,5,1,6,3,185,1,1991,1.0,6.0,2013.0,22.0


#### Forward Selection

In [23]:
### https://stackabuse.com/applying-wrapper-methods-in-python-for-feature-selection/

train_features, test_features, train_labels, test_labels = train_test_split(
    X_train.drop(columns=['ID', 'ID2'], axis=1),
    y_train,
    test_size=0.3,
    random_state=41)

feature_selector = SequentialFeatureSelector(RandomForestClassifier(n_jobs=-1),
           k_features=15,
           forward=True,
           verbose=2,
           cv=4)

features = feature_selector.fit(np.array(train_features.fillna(0)), train_labels)

filtered_features= train_features.columns[list(features.k_feature_idx_)]
filtered_features

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  31 out of  31 | elapsed:  1.4min finished

[2020-08-17 10:17:41] Features: 1/15 -- score: 0.7071070266584425[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.4min finished

[2020-08-17 10:19:04] Features: 2/15 -- score: 0.7589726287029953[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  29 out of  29 | elapsed:  1.4min finished

[2020-08-17 10:20:28] Features: 3/15 -- score: 0.7747971284401843[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

Index(['RIBP', 'SOP4', 'RVSZ', 'PYUQ', 'N2MW', 'BSTQ', 'FM3X', 'K6QO', 'QBOL',
       'JZ9D', 'J9JW', 'ECY3', 'marital_status', 'branch_code', 'date3'],
      dtype='object')

In [24]:
clf = XGBClassifier(learning_rate=0.1675, max_depth=10, gamma=1.976, n_estimators=550)
clf.fit(train_features[filtered_features].fillna(0), train_labels)

train_pred = clf.predict_proba(train_features[filtered_features].fillna(0))
print('AUC on training set: {}'.format(roc_auc_score(train_labels, train_pred[:,1])))

test_pred = clf.predict_proba(test_features[filtered_features].fillna(0))
print('AUC on test set: {}'.format(roc_auc_score(test_labels, test_pred [:,1])))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/agoyal/miniconda3/envs/jupyter/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-c358a04b24e2>", line 2, in <module>
    clf.fit(train_features[filtered_features].fillna(0), train_labels)
  File "/home/agoyal/miniconda3/envs/jupyter/lib/python3.6/site-packages/xgboost/sklearn.py", line 824, in fit
    callbacks=callbacks)
  File "/home/agoyal/miniconda3/envs/jupyter/lib/python3.6/site-packages/xgboost/training.py", line 212, in train
    xgb_model=xgb_model, callbacks=callbacks)
  File "/home/agoyal/miniconda3/envs/jupyter/lib/python3.6/site-packages/xgboost/training.py", line 75, in _train_internal
    bst.update(dtrain, i, obj)
  File "/home/agoyal/miniconda3/envs/jupyter/lib/python3.6/site-packages/xgboost/core.py", line 1369, in update
    dtrain.handle))
KeyboardInterrupt

During handling of the above exception, anothe

TypeError: object of type 'NoneType' has no len()

### Model

In [13]:
# model = CatBoostClassifier()
# parameters = {'depth'         : sp_randInt(4, 10),
#               'learning_rate' : sp_randFloat(),
#               'iterations'    : sp_randInt(10, 100)
#              }

# randm = RandomizedSearchCV(estimator=model, param_distributions = parameters,
#                            cv = 2, n_iter = 10, n_jobs=-1)
# randm.fit(X_train.drop(columns=['ID', 'ID2']), y_train, cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code'])

# # Results from Random Search
# print("\n========================================================")
# print(" Results from Random Search " )
# print("========================================================")
# print("\n The best estimator across ALL searched params:\n",randm.best_estimator_)
# print("\n The best score across ALL searched params:\n",randm.best_score_)
# print("\n The best parameters across ALL searched params:\n",randm.best_params_)
# print("\n ========================================================")
# model = CatBoostClassifier()
# model.fit(X_train.drop(columns=['ID', 'ID2']), y_train, cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code'])

In [14]:
# model = CatBoostClassifier(depth= 7, iterations= 79, learning_rate= 0.24800286296598373)
# model.fit(X_train.drop(columns=['ID', 'ID2']), y_train, cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code'])

In [15]:
# def report_perf(optimizer, X, y, title, callbacks=None):
#     """
#     A wrapper for measuring time and performances of different optmizers
    
#     optimizer = a sklearn or a skopt optimizer
#     X = the training set 
#     y = our target
#     title = a string label for the experiment
#     """
#     start = time()
#     if callbacks:
#         optimizer.fit(X, y, callback=callbacks, cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code'])
#     else:
#         optimizer.fit(X, y, cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code'])
#     d=pd.DataFrame(optimizer.cv_results_)
#     best_score = optimizer.best_score_
#     best_score_std = d.iloc[optimizer.best_index_].std_test_score
#     best_params = optimizer.best_params_
#     print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
#            +u"\u00B1"+" %.3f") % (time() - start, 
#                                   len(optimizer.cv_results_['params']),
#                                   best_score,
#                                   best_score_std))    
#     print('Best parameters:')
#     pprint.pprint(best_params)
#     print()
#     return best_params

In [16]:
# roc_auc = make_scorer(roc_auc_score, greater_is_better=True, needs_threshold=True)
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [17]:
# clf = CatBoostClassifier(cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code'])

In [18]:
# search_spaces = {'iterations': Integer(10, 1000),
#                  'depth': Integer(1, 8),
#                  'learning_rate': Real(0.01, 1.0, 'log-uniform'),
#                  'random_strength': Real(1e-9, 10, 'log-uniform'),
#                  'bagging_temperature': Real(0.0, 1.0),
#                  'border_count': Integer(1, 255),
#                  'l2_leaf_reg': Integer(2, 30),
#                  'scale_pos_weight':Real(0.01, 1.0, 'uniform')}

In [19]:
# # Setting up BayesSearchCV
# opt = BayesSearchCV(clf,
#                     search_spaces,
#                     scoring=roc_auc,
#                     cv=skf,
#                     n_iter=100,
#                     n_jobs=1,  # use just 1 job with CatBoost in order to avoid segmentation fault
#                     return_train_score=False,
#                     refit=True,
#                     optimizer_kwargs={'base_estimator': 'GP'},
#                     random_state=42)

In [20]:
### Random Forest

# rf = RandomForestClassifier(
#     n_estimators=421, 
#     max_depth=15,
#     max_features=3,
#     min_samples_split=8, 
#     class_weight="balanced",
#     bootstrap=True,
#     criterion='entropy',
#     random_state=100
#     )

# rf.fit(X_train.drop(columns=['ID', 'ID2']), y_train)
# preds_rf = rf.predict_proba(X_test.drop(columns=['ID','ID2'], axis=1))
# y_test = pd.DataFrame(preds_rf)
# y_test.columns = le.inverse_transform(y_test.columns)

In [ ]:
# import numpy as np
# from xgboost import XGBClassifier
# from bayes_opt import BayesianOptimization
# from sklearn.model_selection import cross_val_score
# pbounds = {
#     'learning_rate': (0.01, 1.0),
#     'min_child_weight': (0, 10),
#     'max_depth': (0, 50),
#     'max_delta_step': (0, 20),
#     'subsample': (0.01, 1.0),
#     'colsample_bytree': (0.01, 1.0),
#     'colsample_bylevel': (0.01, 1.0),
#     'reg_lambda': (1e-9, 1000),
#     'reg_alpha': (1e-9, 1.0),
#     'gamma': (1e-9, 0.5),
#     'min_child_weight': (0, 5),
#     'n_estimators': (50, 700)
# }
# def xgboost_hyper_param(learning_rate,
#                         n_estimators,
#                         max_depth,
#                         subsample,
#                         gamma,
#                         min_child_weight,
#                         max_delta_step,
#                        colsample_bytree,
#                        colsample_bylevel,
#                        reg_lambda,
#                        reg_alpha):
#     max_depth = int(max_depth)
#     n_estimators = int(n_estimators)
#     clf = XGBClassifier(
#         max_depth=max_depth,
#         learning_rate=learning_rate,
#         n_estimators=n_estimators,
#         gamma=gamma,
#         min_child_weight=min_child_weight,
#         colsample_bytree=colsample_bytree,
#         colsample_bylevel=colsample_bylevel,
#         reg_lambda=reg_lambda,
#         reg_alpha=reg_alpha,
#         max_delta_step=max_delta_step
#         )
#     return np.mean(cross_val_score(clf, X_train.drop(columns=['ID', 'ID2'], axis=1), y_train, cv=3))
# optimizer = BayesianOptimization(
#     f=xgboost_hyper_param,
#     pbounds=pbounds,
#     random_state=1,
# )

# optimizer.maximize(init_points=3,n_iter=10)

In [ ]:
## XGboost

xgb_cl = XGBClassifier(learning_rate=0.1675, max_depth=10, gamma=1.976, n_estimators=550)
xgb_cl.fit(X_train[filtered_features].fillna(0), y_train)
preds_xgb = xgb_cl.predict_proba(X_test[filtered_features].fillna(0))
y_test = pd.DataFrame(preds_xgb)
y_test.columns = le.inverse_transform(y_test.columns)

### Submit

In [ ]:
answer_mass = []
for i in range(X_test.shape[0]):
    id = X_test['ID'].iloc[i]
    for c in y_test.columns:
        answer_mass.append([id + ' X ' + c, y_test[c].iloc[i]])

df_answer = pd.DataFrame(answer_mass)
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
    if df_answer['ID X PCODE'].iloc[i] in true_values:
        df_answer['Label'].iloc[i] = 1.0

In [ ]:
df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('submission22.csv', index=False)

Submit